<a href="https://colab.research.google.com/github/ShivenTripathi/GRiD-R2-2020/blob/master/NextTimeStepPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#############################
# Immediate Future Predictor#
#############################

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
% cd /content/drive/My Drive/dataset/nordstrom/attributes

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/dataset/nordstrom/attributes


In [ ]:
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from matplotlib import pyplot
import os
import pandas
import numpy as np
from collections import defaultdict
import csv

def default_value_attr():
  return "blank"

def def_value_plot():
  return []


f = open("list_attr_cloth.txt", "r")
att_type=[]
i=0
for x in f:
  if i>1:
    x=x.split()
    att_type.append(int(x[len(x)-1]))
  i=i+1 
f.close()

num2word=defaultdict(default_value_attr)
f= open("list_attr_cloth.txt", "r")
i=0
for x in f:
  if i>1:
    s=" ".join(x.split()[:-1])
    num2word[i-2]=s
  i+=1
f.close()

def dfGen(day):
  filename=str(day)+"A.txt"
  #LOAD IMG NAMES
  f = open(filename, "r")
  file_names=[]
  i=2
  for x in f:
    if i%3==0:
      file_names.append(x)
    i=i+1  
  f.close()
  
  #LOAD ATTRIBUTES FOR EACH IMAGE
  f = open(filename, "r")
  att_number=[]
  i=1
  for x in f:
    if i%3==0:
      x = x.split(",")
      y=[]
      for j in range(10):
        y.append(int(x[j]))
      att_number.append(y)
    i=i+1  
  f.close()

  #CATEGORISE ATTRIBUTES
  att=[]
  for i in range(len(att_number)):
    a=[]
    for j in range(5):
      f=0
      for k in range(10):
        if att_type[att_number[i][k]]==j+1:
          f=f+1
          if f==1:
            b=att_number[i][k]
          if f==2:
            b=att_number[i][k]
      if f==0:
        b=-1  
      a.append(b)
    att.append(a)

  #FINAL LIST OF  SECOND MOST POPULAR ATTRIBUTES
  att1=[]
  for i in range(5):
    t=[]
    for j in range(len(att)):
      t.append(att[j][i])
    att1.append(t)  

  #CONVERT TO A DATAFRAME
  df = pandas.DataFrame(data={"source": file_names, "att1": att1[0], "att2": att1[1], "att3": att1[2], "att4": att1[3], "att5": att1[4]})
  return df


class PredictAttribute:

  def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	  n_vars = 1 if type(data) is list else data.shape[1]
	  df = DataFrame(data)
	  cols = list()

	  for i in range(n_in, 0, -1):
		  cols.append(df.shift(i))

	  for i in range(0, n_out):
		  cols.append(df.shift(-i))

	  agg = concat(cols, axis=1)

	  if dropnan:
		  agg.dropna(inplace=True)
	  return agg.values

  def predictAttr(self,days=12,attr=1):
    entries=defaultdict(def_value_plot)
    for i in range(1,days+1):
      df=dfGen(i)
      a=(df['att'+str(attr)].value_counts()/len(df.index)).to_frame('counts').transpose()
      for column in a.columns:
        entries[column].append(a.iloc[0][column])

    with open('temp_data.csv', 'w', newline='') as csvfile:
      fieldnames=['day','value']
      writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
      writer.writeheader()
      for key in entries.keys():
        print(num2word[key])
        for i in range(len(entries[key])):
          writer.writerow({'day':i,'value':entries[key][i]})
        break

    series = read_csv('temp_data.csv', header=0, index_col=0)
    values = series.values
    train = self.series_to_supervised(values, n_in=3)
    trainX, trainy = train[:, :-1], train[:, -1]
    model = XGBRegressor(objective='reg:squarederror', n_estimators=100)
    model.fit(trainX, trainy)
    row = values[-3:].flatten()
    yhat = model.predict(asarray([row]))
    print('Input: %s, Predicted: %.3f' % (row, yhat[0]))
  
    os.remove("temp_data.csv")  

In [ ]:
x=PredictAttribute
x.predictAttr(x)

stripe
Input: [0.54950495 0.5797619  0.5625    ], Predicted: 0.539
